# Pulse propagation in gas-filled hollow core fiber (HCF)

## Manual fitting with spectrum

In [ ]:
# notebook.ipynb
from libraries2 import *
from variables2 import *
from functions2 import *

frequency_FWHM = wavelength_FWHM_nm * speedoflight_nmpfs / wavelength0_nm**2
duration_FWHM_fs = 0.44 /  frequency_FWHM
duration_FWHM = duration_FWHM_fs * 1e-15
wavelength_frontend_nm = np.arange(730, 1331, 1)
spectrum_amplitude_frontend = getGaussianWavelengthSpectrum(wavelength_frontend_nm, wavelength0_nm, wavelength_FWHM_nm)
I_frontend = getPower(spectrum_amplitude_frontend)
wavelength0_m = wavelength0_nm * 1e-9
wavelength_frontend_m = wavelength_frontend_nm * 1e-9
wavelength_FWHM_m = wavelength_FWHM_nm * 1e-9
frequency0 = speedoflight_mps/wavelength0_m                       
omega0 = 2*np.pi*frequency0
attenuation = (1 / z) * np.log(average_power_in/average_power_out)
refractive_index = calculate_refractive_index(pressure)
beta0 = refractive_index * (omega0 / speedoflight_mps)
gamma_calculated = calculate_gamma(n2_atm, pressure, wavelength0_m, core_diameter) 
dz = z / nsteps
Time_window = 15 * duration_FWHM_fs * 1e-15
t = np.linspace(-Time_window/2,Time_window/2,N)                                                                                  
dt = abs(t[1] - t[0])                                   
f = fftshift(fftfreq(N,d=dt))
f_rel = f + frequency0
omega_rel = 2 * np.pi * f_rel
wavelength_rel = speedoflight_mps / f_rel
sort_idx = np.argsort(wavelength_rel)
wavelength_rel = wavelength_rel[sort_idx]
inv_jacobian = (speedoflight_mps) / (wavelength_rel**2)
jacobian = (wavelength_rel**2) / (speedoflight_mps)
interp_func = interp1d(wavelength_frontend_m, I_frontend, kind='cubic', fill_value=0, bounds_error=False)
I_frontend_simgrid = interp_func(wavelength_rel)
I_frontend_simgrid /= np.max(I_frontend_simgrid)
#I_frontend_simgrid = np.clip(I_frontend_simgrid, 0, None)
data = np.loadtxt(filepath, delimiter=";", skiprows=1)
wavelength_spec_ref_nm = data[:, 0]
wavelength_spec_ref_m = wavelength_spec_ref_nm * 1e-9
I_spec_ref = data[:, 2]
interp_func = interp1d(wavelength_spec_ref_m, I_spec_ref, kind='cubic', fill_value=0, bounds_error=False)
I_spec_ref_simgrid = interp_func(wavelength_rel)
I_spec_ref_simgrid /= np.max(I_spec_ref_simgrid)

# Create figure and axes
fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.55)  # Extra space for sliders

# Initial parameter values
gamma_init = gamma_calculated
beta2_init = beta2
beta3_init = beta3
f_R_init = f_R
chirp_init = chirp
FWHM_init = duration_FWHM
alpha_init = alpha
flags_init = [True, True, True, True]       
 
def Simulation(gammavariable, beta2variable, beta3variable, f_R_variable, chirp_variable, FWHM_variable, alphavariable, enable_flags):
    use_nonlinear, use_dispersion, use_chirp, use_FWHM = enable_flags
    if use_chirp:
        chirp = chirp_variable
    else:
        chirp = 0
    duration = FWHM_variable / (2 * np.sqrt(np.log(2)))
    pulse_energy = average_power_in / repetition_frequency             
    peak_power = pulse_energy / duration_FWHM                      
    amplitude = np.sqrt(peak_power)
    A = chirpedGaussianPulseTime(t, amplitude, duration, chirp)
    '''
    A_frontend_wavelength = getGaussianWavelengthSpectrum(wavelength_rel, wavelength0_m, wavelength_FWHM_nm * 1e-9)
    I_frontend_wavelength = getPower(A_frontend_wavelength)
    I_frontend_frequency = I_frontend_wavelength * inv_jacobian
    phase = chirp_variable * (omega_rel - omega0)**2
    # --- Chirp ---
    if use_chirp:
        phase = chirp_variable * (omega_rel - omega0)**2
    else:
        phase = 0.0
    A_frontend_frequency = np.sqrt(I_frontend_frequency) * np.exp(1j * phase)
    A_frontend = getPulseFromSpectrum(t,A_frontend_frequency)
    A_frontend /= np.max(A_frontend)
    A = amplitude * A_frontend
    '''
    for n in range(0,nsteps-1):
        A1 = A.copy()
        #A1 = RK4(A1, attenuation, dz, beta2variable, beta3variable, t, gammavariable, omega0, f_R_variable)
        A1 = FRK4(A1, attenuation, dz, beta2variable, beta3variable, t, gammavariable, omega0, f_R_variable, alphavariable, beta0)
        A = A1 
    A_final = A
    A_spectrum_final = getSpectrumFromPulse(t, A_final)
    I_sim = getPower(A_spectrum_final)
    I_sim /= np.max(I_sim)
    I_sim *= jacobian
    I_sim /= np.max(I_sim)
    print(f"γ = {gammavariable:.5e}, β2​ = {beta2variable:.2e}, β₃ = {beta3variable:.2e}, f_R = {f_R_variable}, chirp = {chirp_variable:.2e}, FWHM = {FWHM_variable:.2e}, alpha = {alphavariable:.2e}")
    return I_sim

# --- Initial plot ---
spectrum = Simulation(gamma_init, beta2_init, beta3_init, f_R_init, chirp_init, FWHM_init, alpha_init, flags_init)
l, = ax.plot(wavelength_rel * 1e9, spectrum, label='Simulated')
l2, = ax.plot(wavelength_rel * 1e9, I_spec_ref_simgrid, '--', label='Measured')

# R² text placeholder
r2_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=10, verticalalignment='top')

ax.set_xlabel("Wavelength [nm]")
ax.set_ylabel("Normalized Intensity")
ax.set_title("Laser pulse propagation in gas-filled HCF")
ax.grid(True)
ax.set_ylim(0, 1)
ax.set_xlim(800, 1200)
ax.legend()

# --- UI control containers ---
ui_controls = {}

# --- Checkbox labels and initial states ---
check_labels = ['Nonlinear', 'Disp & att', 'Chirp', 'FWHM']
checkbox_states = [True, True, True, True]  # All effects enabled by default

# --- Axes for sliders ---
ax_log_gamma = plt.axes([0.25, 0.40, 0.65, 0.03])
ax_beta2 = plt.axes([0.25, 0.35, 0.65, 0.03])
ax_beta3 = plt.axes([0.25, 0.30, 0.65, 0.03])
ax_f_R = plt.axes([0.25, 0.25, 0.65, 0.03])
ax_chirp = plt.axes([0.25, 0.20, 0.65, 0.03])
ax_FWHM = plt.axes([0.25, 0.15, 0.65, 0.03])
ax_alpha = plt.axes([0.25, 0.10, 0.65, 0.03])

# --- Sliders ---
slider_log_gamma = Slider(ax_log_gamma, 'log₁₀(γ)', np.log10(gamma_init), np.log10(1e-1), valinit=np.log10(gamma_init), valstep=0.01)
slider_beta2 = Slider(ax_beta2, 'β₂', -1e-27, 1e-27, valinit=beta2_init, valstep=1e-29)
slider_beta3 = Slider(ax_beta3, 'β₃', -1e-41, 1e-41, valinit=beta3_init, valstep=1e-43)
slider_f_R = Slider(ax_f_R, 'f_R', 0, 1, valinit=f_R_init, valstep=0.01)
slider_chirp = Slider(ax_chirp, 'Chirp', -100, 100, valinit=chirp_init, valstep=1)
slider_FWHM = Slider(ax_FWHM, 'FWHM', 100e-15, 500e-15, valinit=FWHM_init, valstep=1e-15)
slider_alpha = Slider(ax_alpha, 'alpha', 0.998, 1.0, valinit=alpha_init, valstep=0.0001)

# === Helper: Add arrow buttons and register in control dict ===
def add_slider_with_buttons(y_pos, slider, name, step):
    ax_left = plt.axes([0.05, y_pos, 0.04, 0.03])
    ax_right = plt.axes([0.10, y_pos, 0.04, 0.03])
    btn_left = Button(ax_left, '←')
    btn_right = Button(ax_right, '→')

    def left(event):
        val = max(slider.val - step, slider.valmin)
        slider.set_val(val)

    def right(event):
        val = min(slider.val + step, slider.valmax)
        slider.set_val(val)

    btn_left.on_clicked(left)
    btn_right.on_clicked(right)

    # Store all UI elements in a dictionary for easy enabling/disabling
    ui_controls[name] = {
        'slider': slider,
        'buttons': [btn_left, btn_right],
        'axes': [ax_left, ax_right]
    }

# --- Add sliders and buttons to control dict ---
add_slider_with_buttons(0.40, slider_log_gamma, 'SPM', 0.001)
add_slider_with_buttons(0.35, slider_beta2, 'beta2', 1e-30)
add_slider_with_buttons(0.30, slider_beta3, 'beta3', 1e-44)
add_slider_with_buttons(0.25, slider_f_R, 'Raman', 0.001)
add_slider_with_buttons(0.20, slider_chirp, 'Chirp', 0.1)
add_slider_with_buttons(0.15, slider_FWHM, 'FWHM', 1e-16)
add_slider_with_buttons(0.10, slider_alpha, 'alpha', 0.00001)

# === Checkbox setup ===
#checkbox_ax = plt.axes([0.025, 0.65, 0.12, 0.15])
#checkbox = CheckButtons(checkbox_ax, check_labels, checkbox_states)

# --- Update function ---
def update(val):
    gammaval = 10 ** slider_log_gamma.val
    beta2val = slider_beta2.val
    beta3val = slider_beta3.val
    f_R_val = slider_f_R.val
    chirp_val = slider_chirp.val
    FWHM_val = slider_FWHM.val
    alpha_val = slider_alpha.val

    enable_flags = checkbox_states.copy()

    # Run simulation
    new_spec = Simulation(gammaval, beta2val, beta3val, f_R_val, chirp_val, FWHM_val, alpha_val, enable_flags)
    l.set_ydata(new_spec)

    # Compute R²
    y_true = I_spec_ref_simgrid
    y_pred = new_spec
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)

    new_data = {
        'gamma': gammaval,
        'beta2': beta2val,
        'beta3': beta3val,
        'Raman': f_R_val,
        'chirp': chirp_val,
        'FWHM': FWHM_val,
        'alpha_before': alpha_val,
        'r_squared_before': r_squared,
        'alpha_after': 0,
        'r_squared_after': 0,
    }
    df = pd.DataFrame([new_data])
    df.to_csv("state.csv", index=False)

    # Update title and R² text
    #ax.set_title(f"γ = {gammaval:.2e}, β₂ = {beta2val:.2e}, β₃ = {beta3val:.2e}, f_R = {f_R_val}, chirp = {chirp_val:.2e}, FWHM = {FWHM_val:.2e}")
    r2_text.set_text(f"R² = {r_squared:.5f}")
    fig.canvas.draw_idle()

def checkbox_callback(label):
    index = check_labels.index(label)
    checkbox_states[index] = not checkbox_states[index]
    active = checkbox_states[index]

    if label == 'Disp & att':
        for key in ['beta2', 'beta3']:
            ctrl = ui_controls.get(key)
            if ctrl:
                ctrl['slider'].ax.set_visible(active)
                for ax in ctrl['axes']:
                    ax.set_visible(active)
    
    elif label == 'Nonlinear':
        for key in ['SPM', 'Raman']:
            ctrl = ui_controls.get(key)
            if ctrl:
                ctrl['slider'].ax.set_visible(active)
                for ax in ctrl['axes']:
                    ax.set_visible(active)

    # General handling for the rest
    elif label in ui_controls:
        ctrl = ui_controls[label]
        ctrl['slider'].ax.set_visible(active)
        for ax in ctrl['axes']:
            ax.set_visible(active)

    fig.canvas.draw_idle()
    update(None)

# --- Slider callbacks ---
slider_log_gamma.on_changed(update)
slider_beta2.on_changed(update)
slider_beta3.on_changed(update)
slider_f_R.on_changed(update)
slider_chirp.on_changed(update)
slider_FWHM.on_changed(update)
slider_alpha.on_changed(update)

#checkbox.on_clicked(checkbox_callback)

# Show plot
plt.show()
# === END SLIDER UI WITH R² ===

## Computer-aided fitting with spectrum

In [ ]:
df = pd.read_csv('state.csv')
row = df.iloc[0].tolist()
gamma_fitted = row[0]
beta2_fitted = row[1]
beta3_fitted = row[2]
f_R_fitted = row[3]
chirp_fitted = row[4]
FWHM_fitted = row[5]
alpha_before = row[6]
r_squared_before = row[7]

# --- Loss function: spectral L2 distance ---
def spectral_loss(alphatofit):
    duration = FWHM_fitted / (2 * np.sqrt(np.log(2)))
    pulse_energy = average_power_in / repetition_frequency             
    peak_power = pulse_energy / duration_FWHM                      
    amplitude = np.sqrt(peak_power)
    A = chirpedGaussianPulseTime(t, amplitude, duration, chirp_fitted)
    for n in range(0,nsteps-1):
        A1 = A.copy()
        A1 = FRK4(A1, attenuation, dz, beta2_fitted, beta3_fitted, t, gamma_fitted, omega0, f_R_fitted, alphatofit, beta0)
        A = A1 
    A_final = A
    A_spectrum_final = getSpectrumFromPulse(t, A_final)
    I_sim = getPower(A_spectrum_final)
    I_sim /= np.max(I_sim)
    I_sim *= jacobian
    I_sim /= np.max(I_sim)
    return np.sum((I_sim - I_spec_ref_simgrid)**2)

# --- Loss function: spectral L2 distance ---
def spectral_loss2(alphatofit):
    duration = FWHM_fitted / (2 * np.sqrt(np.log(2)))
    pulse_energy = average_power_in / repetition_frequency             
    peak_power = pulse_energy / duration_FWHM                      
    amplitude = np.sqrt(peak_power)
    A = chirpedGaussianPulseTime(t, amplitude, duration, chirp_fitted)
    for n in range(0,nsteps-1):
        A1 = A.copy()
        A1 = FRK4(A1, attenuation, dz, beta2_fitted, beta3_fitted, t, gamma_fitted, omega0, f_R_fitted, alphatofit, beta0)
        A = A1 
    A_final = A
    A_spectrum_final = getSpectrumFromPulse(t, A_final)
    I_sim = getPower(A_spectrum_final)
    I_sim /= np.max(I_sim)
    I_sim *= jacobian
    I_sim /= np.max(I_sim)
    return np.sum((I_sim - np.mean(I_spec_ref_simgrid))**2)

# --- Estimate alpha ---
res = minimize_scalar(spectral_loss, bounds=(0.999, 1.0), method='bounded')
alpha_after = res.x
print(f"Fitted alpha: {alpha_after}")

# Residual sum of squares
ss_res = spectral_loss(alpha_after)
# Total sum of squares
ss_tot = spectral_loss2(alpha_after)
# R^2 value
r_squared_after = 1 - (ss_res / ss_tot)
print(f"R²: {r_squared_after}")

new_data = {
    'gamma': gamma_fitted,
    'beta2': beta2_fitted,
    'beta3': beta3_fitted,
    'Raman': f_R_fitted,
    'chirp': chirp_fitted,
    'FWHM': FWHM_fitted,
    'alpha_before': alpha_before,
    'r_squared_before': r_squared_before,
    'alpha_after': alpha_after,
    'r_squared_after': r_squared_after,
}
df = pd.DataFrame([new_data])
df.to_csv("state.csv", index=False)